# HW07
This homework is due Saturday April 20, 2019 at 11:59pm

## Learning Objective:
1. Students use python version of ITK to quickly review a large number of scan sessions for quality assurance.

## Validate algorithm that Finds Eyes in images.

An algorithm exists that finds many fiducial points (landmark points) in a human MRI brain scan.  The most common failure of this algorithm is an inability to find the Eye lanndmark points in the first stage of the algorithm.  We currently have over 12,000 brain scans processed across 7 different projects.

We need to visually inspect eveery data set to evaluate if the eyes are properly identified with a landmark point approimately in the center of the eye (any landmark that is near the middle of the eye is acceptable.)

The eyes are labeled as `RE` for the right eye and `LE` fro the left eye.  The eyes are approximately `30mm` diameter.  The images do not all have the spaceing for the sampling.

You are tasked with creating a simple HTML report that can be manually reviewed to identify bad data sets.  The 800+ data sets are located at `/nfsscratch/opt/ece5490/data/MIATT_EYES`.  The organization of the files is 

```
   ./${SUBJECT}/${SUBJECT}_${SESSION}/ACPCAlign/BCD_ACPC_Landmarks.fcsv          # A plane ASCII text file with landmark information
   ./${SUBJECT}/${SUBJECT}_${SESSION}/ACPCAlign/Cropped_BCD_ACPC_Aligned.nii.gz  # The image
```

The Landmark files are simple ASCII files that can be easily read with the pandas toolikt

```
# Markups fiducial file version = 4.6
# CoordinateSystem = 0
# columns = id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID
vtkMRMLMarkupsFiducialNode_0,0,0,0,0,0,0,1,1,1,0,AC,,
vtkMRMLMarkupsFiducialNode_1,0.00009494799228537687,-30.201098915930984,-44.52037286965537,0,0,0,1,1,1,0,BPons,,
```

1. The HTML report should have 2D RGB snapshot images written in `.png` format of a region around the eye (1 image for left eye, 1 image for right eye) at the location of the eye fiducial.
1. Rescale each image into a range of values from 0-255.
1. Use an identity transform to resample the original image into a physical space with origin at ( $LE_{L} - 15mm$, $LE_{P}-15mm$, $LE_{S}$ ), with isotropic voxels and a $30mm \times 30mm \times 1voxel$ field of view, and identity direction cosing.  (Remember $_{LPS}$ stands for the DICOM standard $L$eft, $P$osterior, $S$uperior designations This image should be an unsiged char greyscale scalar image.  
1. Convert the 3D image into a 2D image (https://itk.org/Doxygen/html/classitk_1_1ExtractImageFilter.html) that is again a scalar image.
1. Convert the 2D scalar image into a 3D RGB imaage where the R,G, &B channels are all the same values
1. Change the pixel at the index location to pure Red for the Right Eye, and pure Blue for the Left Eye
1. Develop a heuristic correctly identifies at least 1 failed landmark identification correctly (true negative), and no more than 20 correctly identified labels as failed (false negative).  The heuristic results should be listed in the report.
1. Manually review the report and make a list of subjects with Left Eye failures with 1 subject id per line in `failed_le.list`.
1. Manually review the report and make a list of subjects with Right Eye failures with 1 subject id per line in `failed_re.list`.

#### Imports

In [1]:
import itk
import sys
import glob
import os
import shutil
import pandas as pd
import numpy as np
import random
import math
from itkwidgets import view
from matplotlib import pyplot as plt
import pprint
import SimpleITK as sitk
import pickle

#### Fiducials we wish to report on

In [12]:
FIDUCIALS = ['LE', 'RE']

#### Global Configs

In [3]:
IMG_SAVE_DIR = "img_viz"
IMG_VIZ_EXT = "png"
EYE_DIR = '/Volumes/easystore/data/miatt/'
IMAGE_DISP_SIZE = 30  # 30x30 image will be produced for each axis
FID_OFFSET = int(IMAGE_DISP_SIZE/2)

In [4]:
if not os.path.isdir(IMG_SAVE_DIR):
    os.mkdir(IMG_SAVE_DIR)

### Read in specified fiducial points from a given fcsv file

In [5]:
def read_fiducial_points(fcsv_file: str, fiducials:list) -> dict:
    col_names="id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID"
    with open(fcsv_file, 'r') as fid: 
        df = pd.read_csv(fid, sep=',', comment='#', names=col_names.split(',')) 

    fid_pts = {}
    for fid in fiducials:
        fid_pts[fid] = df[ df['label'] == fid].iloc[0][1:4].values
        # flip RAS fiducials to be LPS
        fid_pts[fid][0] = -1*fid_pts[fid][0]
        fid_pts[fid][1] = -1*fid_pts[fid][1]
    return fid_pts

In [6]:
def resample_isotropic(img):
    resampleFilter = sitk.ResampleImageFilter()
    transform = sitk.Transform(3, sitk.sitkIdentity)    
    resampleFilter.SetTransform(transform)
    resampleFilter.SetInterpolator(sitk.sitkLinear)
    resampleFilter.SetSize(img.GetSize())
    resampleFilter.SetOutputSpacing([1,1,1])
    resampleFilter.SetOutputOrigin(img.GetOrigin())
    resampleFilter.SetOutputDirection(np.identity(3).flatten())
    return resampleFilter.Execute(img)

In [7]:
def make_unique_id(subject_id: str, fiducial: str, ext=IMG_VIZ_EXT) -> str:
    if ext is None:
        return "subj_{sub_id}_{fid}".format(sub_id=subject_id, fid=fiducial)
    else:
        return "subj_{sub_id}_{fid}.{ext}".format(sub_id=subject_id, fid=fiducial, ext=ext)

In [8]:
def extract_slice(img, fid_point, axis):
    # bounds checking
    img_size = img.GetSize()
    bounds_keys = ['x','y','z']
    max_bounds = {}
    min_bounds = {}
    
    # fill initial bounds values
    for dim in range(3):
        min_bounds[bounds_keys[dim]] = fid_point[dim]-FID_OFFSET
        max_bounds[bounds_keys[dim]] = fid_point[dim]+FID_OFFSET

        
    # check edge cases
    failed_bounds = []
    for dim in range(3):
        if fid_point[dim]-FID_OFFSET < 0:
            min_bounds[bounds_keys[dim]] = 0
        if fid_point[dim]+FID_OFFSET > img_size[dim] - 1:
            max_bounds[bounds_keys[dim]] = img_size[dim] - 1

    # do the actual extraction
    val = None
    if axis == 0:
        val = img[
            fid_point[0], 
            min_bounds[bounds_keys[1]]:max_bounds[bounds_keys[1]], 
            min_bounds[bounds_keys[2]]:max_bounds[bounds_keys[2]]]

    if axis == 1:
        val = img[
            min_bounds[bounds_keys[0]]:max_bounds[bounds_keys[0]], 
            fid_point[1], 
            min_bounds[bounds_keys[2]]:max_bounds[bounds_keys[2]]]

    if axis == 2:
        val = img[
            min_bounds[bounds_keys[0]]:max_bounds[bounds_keys[0]], 
            min_bounds[bounds_keys[1]]:max_bounds[bounds_keys[1]], 
            fid_point[2]]
    
    # val = np.array(val, dtype=np.uint8)
    val = sitk.GetArrayFromImage(val).astype(np.uint8)
    # adjust for edges from bounds checking
    out = np.zeros((IMAGE_DISP_SIZE, IMAGE_DISP_SIZE), dtype=np.uint8)
    out[:val.shape[0], :val.shape[1]] = val
    return out

In [9]:
RED=(255,0,0)
GREEN=(0,255,0)
BLUE=(0,0,255)

def make_image_viz(image_fn: str, fiducial: str, point, sub_name: str) -> None:
    # read in the image
    image = sitk.ReadImage(image_fn)
    
    # preprocess the image
    rescaler = sitk.RescaleIntensityImageFilter()
    rescaler.SetOutputMinimum(0)
    rescaler.SetOutputMaximum(255)
    image = rescaler.Execute(image)
    image = resample_isotropic(image)

    # extract slices
    fid_ind = image.TransformPhysicalPointToIndex(point)

#     print(fid_ind, image.GetSize())
    x_slice = extract_slice(image, fid_ind, 0)
    y_slice = extract_slice(image, fid_ind, 1)
    z_slice = extract_slice(image, fid_ind, 2)
    
    # convert slices to RGB
    x_slice = np.dstack((x_slice, x_slice, x_slice))
    y_slice = np.dstack((y_slice, y_slice, y_slice))
    z_slice = np.dstack((z_slice, z_slice, z_slice))
    
#     print(x_slice.shape)
#     print(y_slice.shape)
#     print(z_slice.shape)
    
    # set fid point color in each slice
    if fiducial == 'LE':
        color = BLUE
    elif fiducial == 'RE':
        color = RED
    else:
        color = GREEN
        
    x_slice[FID_OFFSET][FID_OFFSET] = color
    y_slice[FID_OFFSET][FID_OFFSET] = color
    z_slice[FID_OFFSET][FID_OFFSET] = color

    all_slice = np.hstack((x_slice, y_slice, z_slice))
#     plt.imshow(all_slice)
#     plt.show()
    
    # save viz
    fid_dir = "{img_save_dir}/{fid}/".format(img_save_dir=IMG_SAVE_DIR, fid=fiducial)
    viz_path = os.path.join(fid_dir, make_unique_id(sub_name, fiducial))
    if not os.path.isdir(fid_dir):
        os.mkdir(fid_dir)
    plt.imsave(viz_path, all_slice) #np.random.random((5,5)))
    return viz_path

In [10]:
with open("./classifier.pickle",'rb') as f:
    clf = pickle.load(f)
    
def eye_heuristic(sub, fid):
    if fid != "LE" and fid != "RE":
        return "unknown"
    thresh = 90. # this is arbitrarily chosen based on HeuristicTrain.ipynb
    fid_dir = "{img_save_dir}/{fid}/".format(img_save_dir=IMG_SAVE_DIR, fid=fid)
    im = sitk.ReadImage(os.path.join(fid_dir,make_unique_id(sub, fid)))
    arr = sitk.GetArrayFromImage(im)
    avg = np.average(arr.flatten())
    return "Bad" if clf.predict([[avg]]) else "Good"

#### Generate Report for all subjects

In [13]:
all_subject_dirs = glob.glob(os.path.join(EYE_DIR,'*'))
all_subject_ids = [ os.path.basename(sd) for sd in all_subject_dirs]
all_subject_ids.sort()
with open('report.html','w') as htmlfid:
    htmlfid.write("<!DOCTYPE html>\n")
    htmlfid.write("<html>\n")
    htmlfid.write("<head>\n")
    htmlfid.write("<style>\n")
    htmlfid.write("table, td, th {\n")
    htmlfid.write("border: 1px solid black;\n")
    htmlfid.write("}\n")
    htmlfid.write("</style>\n")
    htmlfid.write("</head>\n")
    htmlfid.write('<body>\n')
    htmlfid.write('<h1>{0}</h1>\n'.format("Fiducial Point Report"))
    htmlfid.write('<p>{0}</p>\n'.format("Author: Alexander B. Powers (hawkid=abpwrs)"))
    htmlfid.write('<p>{0}</p>\n'.format("Fiducial Points Evaluate: {0}".format(", ".join(FIDUCIALS))))
    htmlfid.write('<table>\n')
    fids_th_str = ""
    for fid in FIDUCIALS:
        fids_th_str = "{recur}<th>{fid_th}</th>".format(recur=fids_th_str, fid_th=fid)
        
    htmlfid.write('<tr><th> subject </th>{th_fids}'.format(th_fids=fids_th_str))

    for subject in all_subject_ids[:]:
        print("processing: %s" % subject)
        fid_fn = glob.glob(os.path.join(EYE_DIR, subject, '*','*','*.fcsv'))[0]
        img_fn = glob.glob(os.path.join(EYE_DIR, subject, '*','*','*.nii.gz'))[0]
        
        fiducial_dict = read_fiducial_points(fid_fn, FIDUCIALS)
        htmlfid.write('  <tr>\n')
        htmlfid.write('     <td>{0}</td>\n'.format(subject))
        
        
        for fid in FIDUCIALS:
            html_fname = make_image_viz(img_fn, fid, fiducial_dict[fid], subject)
            fid_classification = eye_heuristic(subject, fid)
            htmlfid.write('     <td><img src="{0}" width="200" height="80">{1}</td>\n'.format(html_fname, fid_classification))
        
        htmlfid.write('  </tr>\n')
    htmlfid.write('</table>\n')
    htmlfid.write('</body>\n')
    htmlfid.write('</html>\n')

processing: 0000
processing: 0001
processing: 0002
processing: 0003
processing: 0004
processing: 0005
processing: 0006
processing: 0007
processing: 0008
processing: 0009
processing: 0010
processing: 0011
processing: 0012
processing: 0013
processing: 0014
processing: 0015
processing: 0016
processing: 0017
processing: 0018
processing: 0019
processing: 0020
processing: 0021
processing: 0022
processing: 0023
processing: 0024
processing: 0025
processing: 0026
processing: 0027
processing: 0028
processing: 0029
processing: 0030
processing: 0031
processing: 0032
processing: 0033
processing: 0034
processing: 0035
processing: 0036
processing: 0037
processing: 0038
processing: 0039
processing: 0040
processing: 0041
processing: 0042
processing: 0043
processing: 0044
processing: 0045
processing: 0046
processing: 0047
processing: 0048
processing: 0049
processing: 0050
processing: 0051
processing: 0052
processing: 0053
processing: 0054
processing: 0055
processing: 0056
processing: 0057
processing: 00